In [3]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import root_mean_squared_error, r2_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from catboost import CatBoostRegressor
import optuna



In [3]:
df_tr = pd.read_csv('/Users/dmitrybertnikov/Desktop/ds_learn/work_on_datasets/data/raw/train.csv')
df_tr.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents,accident_risk
0,0,urban,2,0.06,35,daylight,rainy,False,True,afternoon,False,True,1,0.13
1,1,urban,4,0.99,35,daylight,clear,True,False,evening,True,True,0,0.35
2,2,rural,4,0.63,70,dim,clear,False,True,morning,True,False,2,0.30
3,3,highway,4,0.07,35,dim,rainy,True,True,morning,False,False,1,0.21
4,4,rural,1,0.58,60,daylight,foggy,False,False,evening,True,False,1,0.56


In [4]:
df = df_tr.drop(columns='id')

In [ ]:
df.head().T

,0,1,2,3,4
road_type,urban,urban,rural,highway,rural
num_lanes,2,4,4,4,1
curvature,0.06,0.99,0.63,0.07,0.58
speed_limit,35,35,70,35,60
lighting,daylight,daylight,dim,dim,daylight
weather,rainy,clear,clear,rainy,foggy
road_signs_present,False,True,False,True,False
public_road,True,False,True,True,False
time_of_day,afternoon,evening,morning,morning,evening
holiday,False,True,True,False,True


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 517754 entries, 0 to 517753
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   road_type               517754 non-null  object 
 1   num_lanes               517754 non-null  int64  
 2   curvature               517754 non-null  float64
 3   speed_limit             517754 non-null  int64  
 4   lighting                517754 non-null  object 
 5   weather                 517754 non-null  object 
 6   road_signs_present      517754 non-null  bool   
 7   public_road             517754 non-null  bool   
 8   time_of_day             517754 non-null  object 
 9   holiday                 517754 non-null  bool   
 10  school_season           517754 non-null  bool   
 11  num_reported_accidents  517754 non-null  int64  
 12  accident_risk           517754 non-null  float64
dtypes: bool(4), float64(2), int64(3), object(4)
memory usage: 37.5+ MB


In [7]:
df.isna().sum()

road_type                 0
num_lanes                 0
curvature                 0
speed_limit               0
lighting                  0
weather                   0
road_signs_present        0
public_road               0
time_of_day               0
holiday                   0
school_season             0
num_reported_accidents    0
accident_risk             0
dtype: int64

In [8]:
X = df.drop(columns='accident_risk')
y = df['accident_risk']

In [9]:
cat_features = X.select_dtypes(include='object').columns.tolist()
cat_features
cat_features_indices = [i for i, col in enumerate(X.columns) if col in cat_features]
cat_features_indices

[0, 4, 5, 8]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y , test_size=0.25, random_state=42)



In [ ]:
from sklearn.metrics import r2_score
from sklearn.model_selection import cross_val_score
import numpy as np
import pandas as pd

def train_models(X_train, X_test, y_train, y_test, cat_features=None, cv_folds=5):
    """
    Обучает несколько моделей и выбирает лучшую по R²
    """
    
    # Автоматически определяем категориальные признаки, если не указаны
    if cat_features is None:
        cat_features = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
        print(f"📋 Автоопределены категориальные признаки: {cat_features}")
    
    # Получаем индексы категориальных признаков для CatBoost
    cat_features_indices = [i for i, col in enumerate(X_train.columns) if col in cat_features]
    
    print(f"🎯 Категориальные признаки ({len(cat_features_indices)}): {cat_features_indices}")
    
    models = {
        'CatBoost': CatBoostRegressor(
            cat_features=cat_features_indices,
            verbose=False, 
            random_state=42,
            n_estimators=100
        ),
        'RandomForest': RandomForestRegressor(
            n_estimators=100, 
            random_state=42, 
            n_jobs=-1
        ),
        'GradientBoosting': GradientBoostingRegressor(
            n_estimators=100, 
            random_state=42
        ),
    }
    
    results = []  
    best_score = -np.inf
    best_model = None
    best_model_name = ""
    
    print("\n=== СРАВНЕНИЕ МОДЕЛЕЙ ===")
    print("Модель           | CV R²     | Train R²  | Test R²   | Status")
    print("-" * 55)
    
    for name, model in models.items():
        try:
            # Для моделей кроме CatBoost нужно предварительно закодировать категориальные признаки
            if name != 'CatBoost' and cat_features:
                X_train_encoded = X_train.copy()
                X_test_encoded = X_test.copy()
                
                # Кодируем категориальные признаки
                for feature in cat_features:
                    if feature in X_train_encoded.columns:
                        X_train_encoded[feature] = X_train_encoded[feature].astype('category').cat.codes
                        X_test_encoded[feature] = X_test_encoded[feature].astype('category').cat.codes
                
                X_train_to_use = X_train_encoded
                X_test_to_use = X_test_encoded
            else:
                X_train_to_use = X_train
                X_test_to_use = X_test
            
            # Кросс-валидация
            cv_scores = cross_val_score(model, X_train_to_use, y_train, cv=cv_folds, scoring='r2')
            cv_r2 = cv_scores.mean()
            
            # Обучение
            model.fit(X_train_to_use, y_train)
            
            # Предсказания
            y_train_pred = model.predict(X_train_to_use)
            y_test_pred = model.predict(X_test_to_use)
            
            # Метрики
            train_r2 = r2_score(y_train, y_train_pred)
            test_r2 = r2_score(y_test, y_test_pred)
            
            # Сохраняем результаты 
            results.append({
                'Model': name,
                'CV_R2': cv_r2,
                'Train_R2': train_r2,
                'Test_R2': test_r2,  
                'CV_Std': cv_scores.std(),
                'Model_Object': model
            })
            
            # Проверяем на лучшую модель
            if test_r2 > best_score:
                best_score = test_r2
                best_model = model
                best_model_name = name
            
            print(f"{name:15} | {cv_r2:8.4f}  | {train_r2:8.4f}  | {test_r2:8.4f}  | ✅")
            
        except Exception as e:
            print(f"{name:15} | {'-':8}  | {'-':8}  | {'-':8}  | ❌ {str(e)[:20]}...")
            continue
    
    # Создаем DataFrame 
    if results:  # Проверяем, что есть результаты
        results_df = pd.DataFrame(results)
        
        # Сортируем по Test R² 
        results_df = results_df.sort_values('Test_R2', ascending=False).reset_index(drop=True)
        
        print("\n" + "="*55)
        print(f"🎯 ЛУЧШАЯ МОДЕЛЬ: {best_model_name}")
        print(f"📊 Test R²: {best_score:.4f}")
        print("="*55)
        
        return best_model, best_model_name, results_df
    else:
        print("❌ Ни одна модель не была успешно обучена")
        return None, None, None

# Упрощенная версия для быстрого тестирования
def train_models_simple(X_train, X_test, y_train, y_test, cv_folds=5):
    """
    Упрощенная версия без категориальных признаков
    """
    models = {
        'CatBoost': CatBoostRegressor(verbose=False, random_state=42, n_estimators=100),
        'RandomForest': RandomForestRegressor(n_estimators=100, random_state=42, n_jobs=-1),
        'GradientBoosting': GradientBoostingRegressor(n_estimators=100, random_state=42),
    }
    
    results = []
    best_score = -np.inf
    best_model = None
    best_model_name = ""
    
    print("=== СРАВНЕНИЕ МОДЕЛЕЙ ===")
    print("Модель           | CV R²     | Train R²  | Test R²   | Status")
    print("-" * 55)
    
    for name, model in models.items():
        try:
            # Кросс-валидация
            cv_scores = cross_val_score(model, X_train, y_train, cv=cv_folds, scoring='r2')
            cv_r2 = cv_scores.mean()
            
            # Обучение
            model.fit(X_train, y_train)
            
            # Предсказания
            y_train_pred = model.predict(X_train)
            y_test_pred = model.predict(X_test)
            
            # Метрики
            train_r2 = r2_score(y_train, y_train_pred)
            test_r2 = r2_score(y_test, y_test_pred)
            
            results.append({
                'Model': name,
                'CV_R2': cv_r2,
                'Train_R2': train_r2,
                'Test_R2': test_r2,
                'CV_Std': cv_scores.std(),
                'Model_Object': model
            })
            
            if test_r2 > best_score:
                best_score = test_r2
                best_model = model
                best_model_name = name
            
            print(f"{name:15} | {cv_r2:8.4f}  | {train_r2:8.4f}  | {test_r2:8.4f}  | ✅")
            
        except Exception as e:
            print(f"{name:15} | {'-':8}  | {'-':8}  | {'-':8}  | ❌ {str(e)[:20]}...")
            continue
    
    if results:
        results_df = pd.DataFrame(results)
        results_df = results_df.sort_values('Test_R2', ascending=False).reset_index(drop=True)
        
        print("\n" + "="*55)
        print(f"🎯 ЛУЧШАЯ МОДЕЛЬ: {best_model_name}")
        print(f"📊 Test R²: {best_score:.4f}")
        print("="*55)
        
        return best_model, best_model_name, results_df
    else:
        print("❌ Ни одна модель не была успешно обучена")
        return None, None, None

# Проверка данных перед обучением
def check_data(X_train, X_test, y_train, y_test):
    """
    Проверка корректности данных перед обучением
    """
    print("=== ПРОВЕРКА ДАННЫХ ===")
    print(f"X_train shape: {X_train.shape}")
    print(f"X_test shape: {X_test.shape}")
    print(f"y_train shape: {y_train.shape}")
    print(f"y_test shape: {y_test.shape}")
    
    print(f"\nТипы данных в X_train:")
    print(X_train.dtypes)
    
    print(f"\nПропущенные значения:")
    print(f"X_train: {X_train.isnull().sum().sum()}")
    print(f"X_test: {X_test.isnull().sum().sum()}")
    print(f"y_train: {y_train.isnull().sum()}")
    print(f"y_test: {y_test.isnull().sum()}")
    
    # Проверяем, есть ли категориальные признаки
    cat_features = X_train.select_dtypes(include=['object', 'category']).columns
    if len(cat_features) > 0:
        print(f"\n📋 Найдены категориальные признаки: {list(cat_features)}")
        return list(cat_features)
    else:
        print(f"\n✅ Все признаки числовые")
        return None

# Полный процесс с проверкой
def full_training_pipeline(X_train, X_test, y_train, y_test):
    """
    Полный пайплайн обучения с проверкой данных
    """
    # 1. Проверяем данные
    cat_features = check_data(X_train, X_test, y_train, y_test)
    
    # 2. Обучаем модели
    if cat_features:
        print(f"\n🎯 Обучаем с учетом категориальных признаков...")
        best_model, best_name, results_df = train_models(
            X_train, X_test, y_train, y_test, cat_features=cat_features
        )
    else:
        print(f"\n🎯 Обучаем модели...")
        best_model, best_name, results_df = train_models_simple(
            X_train, X_test, y_train, y_test
        )
    
    return best_model, best_name, results_df

# Запуск
print("Запуск исправленной функции...")
best_model, best_name, results_df = full_training_pipeline(X_train, X_test, y_train, y_test)

if results_df is not None:
    print(f"\n📊 Финальные результаты:")
    print(results_df[['Model', 'CV_R2', 'Train_R2', 'Test_R2']].round(4))
else:
    print("❌ Обучение не удалось")

Запуск исправленной функции...
=== ПРОВЕРКА ДАННЫХ ===
X_train shape: (388315, 12)
X_test shape: (129439, 12)
y_train shape: (388315,)
y_test shape: (129439,)

Типы данных в X_train:
road_type                  object
num_lanes                   int64
curvature                 float64
speed_limit                 int64
lighting                   object
weather                    object
road_signs_present           bool
public_road                  bool
time_of_day                object
holiday                      bool
school_season                bool
num_reported_accidents      int64
dtype: object

Пропущенные значения:
X_train: 0
X_test: 0
y_train: 0
y_test: 0

📋 Найдены категориальные признаки: ['road_type', 'lighting', 'weather', 'time_of_day']

🎯 Обучаем с учетом категориальных признаков...
🎯 Категориальные признаки (4): [0, 4, 5, 8]

=== СРАВНЕНИЕ МОДЕЛЕЙ ===
Модель           | CV R²     | Train R²  | Test R²   | Status
-------------------------------------------------------
CatBo

In [12]:
df_test = pd.read_csv('/Users/dmitrybertnikov/Desktop/ds_learn/work_on_datasets/data/raw/test.csv')
df_test.head()

,id,road_type,num_lanes,curvature,speed_limit,lighting,weather,road_signs_present,public_road,time_of_day,holiday,school_season,num_reported_accidents
0,517754,highway,2,0.34,45,night,clear,True,True,afternoon,True,True,1
1,517755,urban,3,0.04,45,dim,foggy,True,False,afternoon,True,False,0
2,517756,urban,2,0.59,35,dim,clear,True,False,afternoon,True,True,1
3,517757,rural,4,0.95,35,daylight,rainy,False,False,afternoon,False,False,2
4,517758,highway,2,0.86,35,daylight,clear,True,False,evening,False,True,3


In [ ]:
# TODO Доделать оптимизацию через оптуну
# def objective_catboost(trial):
#     max_depth = 4 #trial.suggest_int("max_depth", 3, 5)
#     learning_rate = trial.suggest_float("learning_rate", 1e-5, 1, log=True)
#     n_estimators = trial.suggest_int("n_estimators", 1200, 2000)
    

#     model = CatBoostRegressor(
#         cat_features=cat_features_indices,
#         max_depth=max_depth,
#         learning_rate=learning_rate,
#         n_estimators=n_estimators)

    

#     score = cross_val_score(model, X, y, cv=3, scoring="r2", n_jobs=-1).mean()
#     return score

# study = optuna.create_study(direction="maximize")
# study.optimize(objective_catboost, n_trials=30)


[I 2025-10-30 12:22:08,172] A new study created in memory with name: no-name-c1d59af2-cb87-4437-a370-12c6d018b5bb


0:	learn: 0.1664502	total: 171ms	remaining: 4m 51s
0:	learn: 0.1663718	total: 179ms	remaining: 5m 5s
0:	learn: 0.1664201	total: 187ms	remaining: 5m 18s
1:	learn: 0.1664474	total: 259ms	remaining: 3m 40s
1:	learn: 0.1664173	total: 263ms	remaining: 3m 44s
1:	learn: 0.1663689	total: 266ms	remaining: 3m 46s
2:	learn: 0.1664445	total: 339ms	remaining: 3m 12s
2:	learn: 0.1664144	total: 349ms	remaining: 3m 17s
2:	learn: 0.1663661	total: 365ms	remaining: 3m 27s
3:	learn: 0.1664417	total: 405ms	remaining: 2m 52s
3:	learn: 0.1664116	total: 412ms	remaining: 2m 55s
3:	learn: 0.1663632	total: 478ms	remaining: 3m 23s
4:	learn: 0.1664388	total: 513ms	remaining: 2m 54s
4:	learn: 0.1664087	total: 518ms	remaining: 2m 56s
4:	learn: 0.1663604	total: 550ms	remaining: 3m 6s
5:	learn: 0.1664360	total: 557ms	remaining: 2m 37s
5:	learn: 0.1664059	total: 579ms	remaining: 2m 43s
5:	learn: 0.1663575	total: 614ms	remaining: 2m 53s
6:	learn: 0.1664331	total: 644ms	remaining: 2m 36s
6:	learn: 0.1663547	total: 677ms	

[I 2025-10-30 12:23:41,595] Trial 0 finished with value: 0.05622844684440257 and parameters: {'learning_rate': 2.2398695691847742e-05, 'n_estimators': 1706}. Best is trial 0 with value: 0.05622844684440257.


0:	learn: 0.1664024	total: 46.2ms	remaining: 1m 27s
1:	learn: 0.1663519	total: 85.7ms	remaining: 1m 21s
2:	learn: 0.1663013	total: 127ms	remaining: 1m 20s
3:	learn: 0.1662508	total: 153ms	remaining: 1m 12s
4:	learn: 0.1662002	total: 178ms	remaining: 1m 7s
5:	learn: 0.1661498	total: 203ms	remaining: 1m 3s
6:	learn: 0.1660993	total: 239ms	remaining: 1m 4s
7:	learn: 0.1660487	total: 274ms	remaining: 1m 4s
8:	learn: 0.1659984	total: 303ms	remaining: 1m 3s
9:	learn: 0.1659480	total: 327ms	remaining: 1m 1s
10:	learn: 0.1658976	total: 351ms	remaining: 1m
11:	learn: 0.1658472	total: 376ms	remaining: 59s
12:	learn: 0.1657969	total: 402ms	remaining: 58.2s
13:	learn: 0.1657466	total: 429ms	remaining: 57.6s
14:	learn: 0.1656964	total: 458ms	remaining: 57.4s
15:	learn: 0.1656462	total: 486ms	remaining: 57.1s
16:	learn: 0.1655961	total: 513ms	remaining: 56.7s
17:	learn: 0.1655459	total: 539ms	remaining: 56.2s
18:	learn: 0.1654957	total: 564ms	remaining: 55.7s
19:	learn: 0.1654456	total: 591ms	remain

[I 2025-10-30 12:25:34,560] Trial 1 finished with value: 0.6138503816640245 and parameters: {'learning_rate': 0.0003980417176737738, 'n_estimators': 1895}. Best is trial 1 with value: 0.6138503816640245.


...

0:	learn: 0.1664062	total: 95.8ms	remaining: 2m 51s
1:	learn: 0.1663595	total: 181ms	remaining: 2m 41s
0:	learn: 0.1663762	total: 125ms	remaining: 3m 43s
2:	learn: 0.1663127	total: 257ms	remaining: 2m 33s
1:	learn: 0.1663294	total: 185ms	remaining: 2m 45s
3:	learn: 0.1662660	total: 326ms	remaining: 2m 25s
4:	learn: 0.1662192	total: 382ms	remaining: 2m 16s
2:	learn: 0.1662826	total: 280ms	remaining: 2m 46s
5:	learn: 0.1661725	total: 436ms	remaining: 2m 9s
3:	learn: 0.1662359	total: 331ms	remaining: 2m 27s
6:	learn: 0.1661258	total: 525ms	remaining: 2m 13s
4:	learn: 0.1661892	total: 429ms	remaining: 2m 33s
5:	learn: 0.1661424	total: 480ms	remaining: 2m 22s
7:	learn: 0.1660791	total: 599ms	remaining: 2m 13s
8:	learn: 0.1660325	total: 667ms	remaining: 2m 12s
6:	learn: 0.1660958	total: 571ms	remaining: 2m 25s
9:	learn: 0.1659859	total: 726ms	remaining: 2m 9s
7:	learn: 0.1660491	total: 620ms	remaining: 2m 18s
10:	learn: 0.1659393	total: 789ms	remaining: 2m 7s
8:	learn: 0.1660026	total: 681ms

[I 2025-10-30 12:34:05,935] Trial 5 finished with value: 0.5681396917784972 and parameters: {'learning_rate': 0.0003681136464728671, 'n_estimators': 1791}. Best is trial 1 with value: 0.6138503816640245.


0:	learn: 0.1663634	total: 91ms	remaining: 1m 53s
1:	learn: 0.1663521	total: 187ms	remaining: 1m 56s
0:	learn: 0.1664418	total: 124ms	remaining: 2m 34s
2:	learn: 0.1663409	total: 312ms	remaining: 2m 9s
1:	learn: 0.1664306	total: 244ms	remaining: 2m 32s
0:	learn: 0.1664117	total: 131ms	remaining: 2m 44s
3:	learn: 0.1663297	total: 390ms	remaining: 2m 1s
2:	learn: 0.1664193	total: 331ms	remaining: 2m 17s
1:	learn: 0.1664005	total: 221ms	remaining: 2m 18s
4:	learn: 0.1663184	total: 472ms	remaining: 1m 57s
3:	learn: 0.1664081	total: 390ms	remaining: 2m 1s
2:	learn: 0.1663892	total: 317ms	remaining: 2m 12s
5:	learn: 0.1663072	total: 573ms	remaining: 1m 58s
4:	learn: 0.1663968	total: 472ms	remaining: 1m 57s
3:	learn: 0.1663780	total: 385ms	remaining: 2m
6:	learn: 0.1662959	total: 651ms	remaining: 1m 55s
5:	learn: 0.1663856	total: 553ms	remaining: 1m 54s
4:	learn: 0.1663667	total: 473ms	remaining: 1m 57s
7:	learn: 0.1662847	total: 745ms	remaining: 1m 55s
6:	learn: 0.1663743	total: 658ms	remain

[I 2025-10-30 12:35:37,429] Trial 6 finished with value: 0.15192085379255446 and parameters: {'learning_rate': 8.84744772537487e-05, 'n_estimators': 1252}. Best is trial 1 with value: 0.6138503816640245.


0:	learn: 0.1663262	total: 103ms	remaining: 2m 35s
1:	learn: 0.1662779	total: 219ms	remaining: 2m 46s
0:	learn: 0.1664046	total: 161ms	remaining: 4m 3s
0:	learn: 0.1663746	total: 156ms	remaining: 3m 55s
2:	learn: 0.1662295	total: 362ms	remaining: 3m 2s
1:	learn: 0.1663563	total: 297ms	remaining: 3m 45s
1:	learn: 0.1663262	total: 241ms	remaining: 3m 2s
3:	learn: 0.1661812	total: 448ms	remaining: 2m 49s
2:	learn: 0.1663079	total: 410ms	remaining: 3m 26s
2:	learn: 0.1662778	total: 357ms	remaining: 3m
4:	learn: 0.1661329	total: 519ms	remaining: 2m 36s
3:	learn: 0.1662294	total: 428ms	remaining: 2m 41s
3:	learn: 0.1662595	total: 500ms	remaining: 3m 9s
5:	learn: 0.1660846	total: 600ms	remaining: 2m 31s
4:	learn: 0.1662112	total: 593ms	remaining: 2m 59s
4:	learn: 0.1661811	total: 533ms	remaining: 2m 41s
6:	learn: 0.1660363	total: 684ms	remaining: 2m 27s
5:	learn: 0.1661629	total: 674ms	remaining: 2m 49s
7:	learn: 0.1659881	total: 752ms	remaining: 2m 21s
5:	learn: 0.1661327	total: 617ms	remain